In [ ]:
import pandas as pd
import glob
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Path for An
opath = '/content/drive/MyDrive/01. Master Degree/03. Data Mining/Group 8 Project'

In [ ]:
# Path for Owen
opath = '/content/drive/MyDrive/umn/24-25/zfall 25/csci 5523/Group 8 Project'

In [ ]:
# Path for Ahmed
# opath =

In [ ]:
# Path for Nour
# opath =

# 1. Call for merged data

In [ ]:
preprocessed_all = os.path.join(opath, 'Merged Dataframe')

In [ ]:
calming_df = pd.read_csv(os.path.join(preprocessed_all, 'Calming_Merged_feature.csv'))
vexing_df = pd.read_csv(os.path.join(preprocessed_all, 'Vexing_Merged_feature.csv'))
# music_df = pd.read_csv(os.path.join(preprocessed_all, 'Music_df.csv'))

In [ ]:
calming_df.head()

,Unnamed: 0,n_back,Response_Time,condition,subject_id,BlockNumber,TrialInBlock,Correct,BVP_mean,BVP_std,...,ECG_mean,ECG_std,EMG_mean,EMG_std,RESP_mean,RESP_std,PPG_mean,PPG_std,SKT_mean,SKT_std
0,0,1,0.0,Calming,Subject4,1,1,0,-2.751331,5.453607,...,0.010924,0.195181,-0.002213,0.017522,-6.161784,0.105731,0.050346,0.024017,22.968318,0.000834
1,1,1,838.0,Calming,Subject4,1,2,1,4.456672,0.879887,...,0.010689,0.192964,-0.002241,0.017178,-6.033325,0.126568,0.050484,0.019253,22.968283,0.000822
2,2,1,374.0,Calming,Subject4,1,3,1,4.285210,0.740332,...,0.017447,0.221070,-0.002368,0.018552,-6.090992,0.086868,0.050917,0.020832,22.968333,0.000821
3,3,1,405.0,Calming,Subject4,1,4,1,-2.634205,2.918769,...,0.000513,0.216476,-0.002110,0.020708,-5.981885,0.137434,0.046472,0.021422,22.968352,0.000817
4,4,1,39.0,Calming,Subject4,1,5,1,0.772251,0.211038,...,0.006566,0.199935,-0.002105,0.022657,-6.062608,0.227627,0.048821,0.016176,22.968353,0.000816


# 2. Data splitting

In [ ]:
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit

def split_data(df, model_predicting = 'n_back'):
  '''
  In this function, we will split the data into training dataset, validation dataset, and testing dataset
  Args:
  df: Dataframe of raw data
  model_predicting: [str] indicating which model we are trying to split for
  '''

  if model_predicting == 'n_back':
    df_filtered = df[df['n_back'] != 'rest'].copy()
    y = df_filtered['n_back'].astype(int)
    columns_to_drop = [
        'n_back', 'subject_id', 'BlockNumber', 'TrialInBlock',
        'Session', 'Block', 'Trial', 'task' # These are from empatica_df, might not exist in merged df
    ]
    # Ensure only existing columns are dropped
    final_columns_to_drop = [col for col in columns_to_drop if col in df_filtered.columns]
    X = df_filtered.drop(columns=final_columns_to_drop)
    # One-hot encode 'condition' columns
    X = pd.get_dummies(X, columns=['condition'], drop_first=True)

  elif model_predicting == 'condition':
    df_filtered = df[df['n_back'] != 'rest'].copy()
    y = df_filtered['condition'].map({'calming': 0, 'vexing': 1})
    columns_to_drop = ['BlockNumber', 'TrialInBlock', 'subject_id', 'condition', 'task', 'n_back']

    final_columns_to_drop = [col for col in columns_to_drop if col in df_filtered.columns]
    X = df_filtered.drop(columns=final_columns_to_drop)

  groups = df_filtered['subject_id']

  # Split 1: Create the Test set (10% of subjects)
  gss_test = GroupShuffleSplit(n_splits=1, test_size=0.1, random_state=42)
  train_val_idx, test_idx = next(gss_test.split(X, y, groups))

  X_train_val, X_test = X.iloc[train_val_idx], X.iloc[test_idx]
  y_train_val, y_test = y.iloc[train_val_idx], y.iloc[test_idx]
  groups_train_val = groups.iloc[train_val_idx] # Get the subject list for the 90%

  # Split 2: Create the Validation set (10% of total)
  # We take 10% from the remaining 90% (0.10 / 0.90 = ~0.111)
  val_split_size = 0.10 / 0.90
  gss_valid = GroupShuffleSplit(n_splits=1, test_size=val_split_size, random_state=42)
  train_idx, valid_idx = next(gss_valid.split(X_train_val, y_train_val, groups_train_val))

  # Create the final sets
  X_train, X_valid = X_train_val.iloc[train_idx], X_train_val.iloc[valid_idx]
  y_train, y_valid = y_train_val.iloc[train_idx], y_train_val.iloc[valid_idx]

  print("\n Split Verification (80/10/10)")
  print(f"Total rows in X_train: {len(X_train)} (~80%)")
  print(f"Total rows in X_valid: {len(X_valid)} (~10%)")
  print(f"Total rows in X_test:  {len(X_test)} (~10%)")

  print(f"\nTraining subjects:   {groups_train_val.iloc[train_idx].unique()}")
  print(f"Validation subjects: {groups_train_val.iloc[valid_idx].unique()}")
  print(f"Test subjects:       {groups.iloc[test_idx].unique()}")

  # Check for any overlap (should be empty sets)
  train_subs = set(groups_train_val.iloc[train_idx].unique())
  valid_subs = set(groups_train_val.iloc[valid_idx].unique())
  test_subs = set(groups.iloc[test_idx].unique())

  print(f"\nOverlap Train/Valid: {train_subs.intersection(valid_subs)}")
  print(f"Overlap Train/Test:  {train_subs.intersection(test_subs)}")
  print(f"Overlap Valid/Test:  {valid_subs.intersection(test_subs)}")

  return X_train, X_valid, X_test, y_train, y_valid, y_test

In [ ]:
calm_X_train, calm_X_valid, calm_X_test, calm_y_train, calm_y_valid, calm_y_test = split_data(calming_df)
vex_X_train, vex_X_valid, vex_X_test, vex_y_train, vex_y_valid, vex_y_test = split_data(vexing_df)


 Split Verification (80/10/10)
Total rows in X_train: 1056 (~80%)
Total rows in X_valid: 352 (~10%)
Total rows in X_test:  352 (~10%)

Training subjects:   ['Subject6' 'Subject11' 'Subject8']
Validation subjects: ['Subject4']
Test subjects:       ['Subject3']

Overlap Train/Valid: set()
Overlap Train/Test:  set()
Overlap Valid/Test:  set()

 Split Verification (80/10/10)
Total rows in X_train: 1056 (~80%)
Total rows in X_valid: 352 (~10%)
Total rows in X_test:  352 (~10%)

Training subjects:   ['Subject6' 'Subject8' 'Subject11']
Validation subjects: ['Subject4']
Test subjects:       ['Subject3']

Overlap Train/Valid: set()
Overlap Train/Test:  set()
Overlap Valid/Test:  set()


In [ ]:
calm_y_train.head()

,n_back
352,1
353,1
354,1
355,1
356,1


# 3. Feature selection

In [ ]:
def feature_engineering(X_df, y_df, drop_threshold):
  '''
  This function returns a list of features that have correlation > threshold
  Args:
  X_df: X training dataframe
  y_df: y training dataframe

  Returns:
  X_df_feature_engineering: list of index
  '''
  # Computing the correlation of each feature against the label 'n-back'
  X_df_feature_engineering = X_df.corrwith(y_df)

  print("\nFeature correlations:")
  print(abs(X_df_feature_engineering).sort_values(ascending=False))

  # Dropping all features that have correlation < threshold
  X_df_feature_engineering = X_df_feature_engineering[abs(X_df_feature_engineering) > drop_threshold]

  print("\nSelecting features that have correlation > {} with n-back label".format(drop_threshold))
  print(X_df_feature_engineering)
  print(f"\n{'='*40}")
  return X_df_feature_engineering.index.to_list()

In [ ]:
calm_X_train_FE = feature_engineering(calm_X_train, calm_y_train, 0.05)
vex_X_train_FE = feature_engineering(vex_X_train, vex_y_train, 0.03)


Feature correlations:
Correct              0.246588
HR_std               0.124619
Response_Time        0.107886
RESP_mean            0.059623
RESP_std             0.047054
SKT_std              0.039030
ECG_std              0.038174
EDA_std_empatica     0.032715
ACC_std              0.032237
EDA_std_biopac       0.029649
PPG_mean             0.028711
ECG_mean             0.026279
EMG_std              0.023868
BVP_std              0.022212
IBI_std              0.018356
TEMP_mean            0.018070
PPG_std              0.017382
SKT_mean             0.015683
IBI_mean             0.015456
EDA_mean_biopac      0.014946
HR_mean              0.011025
EMG_mean             0.009455
Unnamed: 0           0.009021
EDA_mean_empatica    0.006749
BVP_mean             0.004331
ACC_mean             0.003182
dtype: float64

Selecting features that have correlation > 0.05 with n-back label
Response_Time    0.107886
Correct         -0.246588
HR_std          -0.124619
RESP_mean        0.059623
dtype: floa

In [ ]:
calm_X_retrain = calm_X_train[calm_X_train_FE]
vex_X_retrain = vex_X_train[vex_X_train_FE]

In [ ]:
calm_X_revalid = calm_X_valid[calm_X_train_FE]
vex_X_revalid = vex_X_valid[vex_X_train_FE]

calm_X_retest = calm_X_test[calm_X_train_FE]
vex_X_retest = vex_X_test[vex_X_train_FE]

# 4. XGBoost classifier

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def XGBC_train_and_evaluate_model(X_train, X_valid, X_test, y_train, y_valid, y_test, optimizing_model = False):

  scaler = StandardScaler()
  X_train_scaled = scaler.fit_transform(X_train)

  X_valid_scaled = scaler.transform(X_valid)
  X_test_scaled = scaler.transform(X_test)

  print("Feature scaling complete.")

  ############## OPTIMIZATION CODE - Below #####################
  if optimizing_model == True:
    print("Beginning XGBoost Hyperparameter Optimization")

    param_grid = {
        'n_estimators': [200, 400, 800],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.05, 0.1],
        'subsample': [0.7, 1.0],
        'colsample_bytree': [0.7, 1.0],
        'gamma': [0, 1, 3],
        'reg_alpha': [0, 0.5, 1],
        'reg_lambda': [1, 5]
    }

    xgb_base = xgb.XGBClassifier(
        objective="binary:logistic",
        eval_metric="logloss",
        random_state=42,
        tree_method="hist"
    )

    grid = GridSearchCV(
        estimator=xgb_base,
        param_grid=param_grid,
        scoring='f1',
        cv=3,
        verbose=1,
        n_jobs=-1
    )

    grid.fit(X_train_scaled, y_train)

    print("Best Params:", grid.best_params_)
    print("Best Score:", grid.best_score_)

    # Replace model with optimized version
    model = grid.best_estimator_

  ############## OPTIMIZING CODE - Complete ######################
  else:
    model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss',
                              early_stopping_rounds = 20,
                              random_state=42,reg_alpha=1,reg_lambda=2)

  print("Training the XGBoost model...")
  # Removed 'early_stopping_rounds' to resolve TypeError.
  # Model will now train for full number of boosting rounds.
  model.fit(X_train_scaled, y_train,
            eval_set=[(X_valid_scaled, y_valid)],
            # early_stopping_rounds = 10,
            verbose=False
           )
  print("Training complete.")

  # TRAINING EVALUATION
  y_pred_train = model.predict(X_train_scaled)
  train_acc = accuracy_score(y_train, y_pred_train)
  print("Training Accuracy:", train_acc * 100)

  # VALIDATION EVALUATION
  y_pred_valid = model.predict(X_valid_scaled)
  accuracy = accuracy_score(y_valid, y_pred_valid)

  print("\nModel Evaluation")
  print(f"Validation Accuracy: {accuracy * 100:.2f}%")
  print("Recall:", recall_score(y_valid, y_pred_valid)*100)
  print("F1 Score:", f1_score(y_valid, y_pred_valid)*100)
  print("Confusion Matrix:\n", confusion_matrix(y_valid, y_pred_valid))

  # TEST EVALUATION
  y_pred_test = model.predict(X_test_scaled)
  test_acc = accuracy_score(y_test, y_pred_test)

  print("\nTest Evaluation")
  print(f"Test Accuracy: {test_acc * 100:.2f}%")
  print("Recall:", recall_score(y_test, y_pred_test)*100)
  print("F1 Score:", f1_score(y_test, y_pred_test)*100)
  print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_test))

  print("\nFeature Importance")
  feature_importances = pd.Series(model.feature_importances_, index=X_train.columns).sort_values(ascending=False)
  print(feature_importances)
  return model

In [ ]:
calm_y_train_mapped = calm_y_train.replace({1: 0, 3: 1})
calm_y_valid_mapped = calm_y_valid.replace({1: 0, 3: 1})
calm_y_test_mapped = calm_y_test.replace({1: 0, 3: 1})

vex_y_train_mapped = vex_y_train.replace({1: 0, 3: 1})
vex_y_valid_mapped = vex_y_valid.replace({1: 0, 3: 1})
vex_y_test_mapped = vex_y_test.replace({1: 0, 3: 1})

## XGBoost without optimization

In [ ]:
# XGBoost on calm set
XGBC_train_and_evaluate_model(calm_X_train, calm_X_valid, calm_X_test, calm_y_train_mapped, calm_y_valid_mapped, calm_y_test_mapped, optimizing_model= False)

Feature scaling complete.
Training the XGBoost model...


/usr/local/lib/python3.12/dist-packages/xgboost/callback.py:386: UserWarning: [21:56:43] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  self.starting_round = model.num_boosted_rounds()


Training complete.
Training Accuracy: 89.29924242424242

Model Evaluation
Validation Accuracy: 44.03%
Recall: 51.70454545454546
F1 Score: 48.02110817941953
Confusion Matrix:
 [[ 64 112]
 [ 85  91]]

Test Evaluation
Test Accuracy: 55.40%
Recall: 72.1590909090909
F1 Score: 61.80048661800487
Confusion Matrix:
 [[ 68 108]
 [ 49 127]]

Feature Importance
Correct              0.224404
Response_Time        0.076100
RESP_std             0.068829
HR_std               0.047901
Unnamed: 0           0.044054
EDA_mean_biopac      0.043114
HR_mean              0.036602
SKT_mean             0.035675
EDA_mean_empatica    0.034191
TEMP_mean            0.033582
SKT_std              0.032947
ACC_mean             0.027976
BVP_std              0.027211
ECG_std              0.026616
EDA_std_biopac       0.026135
RESP_mean            0.025159
PPG_std              0.023715
PPG_mean             0.022856
BVP_mean             0.022345
EDA_std_empatica     0.020741
EMG_mean             0.020205
EMG_std           

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=20,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [ ]:
# XGBoost on vex set
XGBC_train_and_evaluate_model(vex_X_train, vex_X_valid, vex_X_test, vex_y_train_mapped, vex_y_valid_mapped, vex_y_test_mapped, optimizing_model= False)

Feature scaling complete.
Training the XGBoost model...


/usr/local/lib/python3.12/dist-packages/xgboost/callback.py:386: UserWarning: [21:56:44] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  self.starting_round = model.num_boosted_rounds()


Training complete.
Training Accuracy: 82.1969696969697

Model Evaluation
Validation Accuracy: 62.50%
Recall: 52.84090909090909
F1 Score: 58.490566037735846
Confusion Matrix:
 [[127  49]
 [ 83  93]]

Test Evaluation
Test Accuracy: 55.40%
Recall: 23.863636363636363
F1 Score: 34.85477178423236
Confusion Matrix:
 [[153  23]
 [134  42]]

Feature Importance
Correct              0.264090
Response_Time        0.078961
ECG_std              0.052299
HR_mean              0.044295
EDA_mean_biopac      0.041940
PPG_std              0.041508
RESP_mean            0.040562
TEMP_mean            0.037807
IBI_mean             0.037676
Unnamed: 0           0.034704
SKT_mean             0.032398
IBI_std              0.029579
EMG_std              0.025108
RESP_std             0.023722
EDA_mean_empatica    0.023396
EDA_std_empatica     0.022153
EDA_std_biopac       0.021326
ACC_mean             0.020716
BVP_std              0.019909
ACC_std              0.019539
HR_std               0.018078
SKT_std         

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=20,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

## XGBoost with optimization and no feature selection

In [ ]:
# XGBoost on calm set
XGBC_train_and_evaluate_model(calm_X_train, calm_X_valid, calm_X_test, calm_y_train_mapped, calm_y_valid_mapped, calm_y_test_mapped, optimizing_model= True)

Feature scaling complete.
Beginning XGBoost Hyperparameter Optimization
Fitting 3 folds for each of 1944 candidates, totalling 5832 fits
Best Params: {'colsample_bytree': 0.7, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 200, 'reg_alpha': 0.5, 'reg_lambda': 5, 'subsample': 1.0}
Best Score: 0.5424479419495655
Training the XGBoost model...
Training complete.
Training Accuracy: 83.52272727272727

Model Evaluation
Validation Accuracy: 63.07%
Recall: 52.84090909090909
F1 Score: 58.86075949367089
Confusion Matrix:
 [[129  47]
 [ 83  93]]

Test Evaluation
Test Accuracy: 72.44%
Recall: 81.25
F1 Score: 74.67362924281984
Confusion Matrix:
 [[112  64]
 [ 33 143]]

Feature Importance
Correct              0.114497
Response_Time        0.093933
EDA_mean_biopac      0.075266
HR_mean              0.068924
EDA_mean_empatica    0.056323
RESP_std             0.054701
HR_std               0.054441
EMG_std              0.050265
TEMP_mean            0.041077
RESP_mean            0.0392

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=0,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, ...)

In [ ]:
# XGBoost on vex set
XGBC_train_and_evaluate_model(vex_X_train, vex_X_valid, vex_X_test, vex_y_train_mapped, vex_y_valid_mapped, vex_y_test_mapped, optimizing_model= True)

Feature scaling complete.
Beginning XGBoost Hyperparameter Optimization
Fitting 3 folds for each of 1944 candidates, totalling 5832 fits
Best Params: {'colsample_bytree': 0.7, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 400, 'reg_alpha': 1, 'reg_lambda': 5, 'subsample': 0.7}
Best Score: 0.42234742229210953
Training the XGBoost model...
Training complete.
Training Accuracy: 100.0

Model Evaluation
Validation Accuracy: 50.57%
Recall: 15.340909090909092
F1 Score: 23.684210526315788
Confusion Matrix:
 [[151  25]
 [149  27]]

Test Evaluation
Test Accuracy: 60.23%
Recall: 71.5909090909091
F1 Score: 64.28571428571429
Confusion Matrix:
 [[ 86  90]
 [ 50 126]]

Feature Importance
Correct              0.184264
Response_Time        0.076594
PPG_std              0.058772
RESP_mean            0.057013
HR_mean              0.045767
EDA_mean_biopac      0.041889
ECG_std              0.039450
TEMP_mean            0.038860
EMG_std              0.038475
RESP_std             0.03775

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=0,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=400, n_jobs=None,
              num_parallel_tree=None, ...)

## XGBoost with optimization and feature selection

In [ ]:
# Optimized XGBoost on calm set
XGBC_train_and_evaluate_model(calm_X_retrain, calm_X_revalid, calm_X_retest, calm_y_train_mapped, calm_y_valid_mapped, calm_y_test_mapped, optimizing_model= True)

Feature scaling complete.
Beginning XGBoost Hyperparameter Optimization
Fitting 3 folds for each of 1944 candidates, totalling 5832 fits
Best Params: {'colsample_bytree': 1.0, 'gamma': 3, 'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 200, 'reg_alpha': 1, 'reg_lambda': 5, 'subsample': 0.7}
Best Score: 0.6268484664378537
Training the XGBoost model...
Training complete.
Training Accuracy: 71.68560606060606

Model Evaluation
Validation Accuracy: 67.61%
Recall: 54.54545454545454
F1 Score: 62.745098039215684
Confusion Matrix:
 [[142  34]
 [ 80  96]]

Test Evaluation
Test Accuracy: 62.78%
Recall: 89.20454545454545
F1 Score: 70.56179775280899
Confusion Matrix:
 [[ 64 112]
 [ 19 157]]

Feature Importance
Correct          0.449825
Response_Time    0.229375
HR_std           0.165079
RESP_mean        0.155721
dtype: float32


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=1.0, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=3,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, ...)

In [ ]:
# Optimized XGBoost on vex set
XGBC_train_and_evaluate_model(vex_X_retrain, vex_X_revalid, vex_X_retest, vex_y_train_mapped, vex_y_valid_mapped, vex_y_test_mapped, optimizing_model= True)

Feature scaling complete.
Beginning XGBoost Hyperparameter Optimization
Fitting 3 folds for each of 1944 candidates, totalling 5832 fits
Best Params: {'colsample_bytree': 1.0, 'gamma': 0, 'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 400, 'reg_alpha': 0.5, 'reg_lambda': 1, 'subsample': 1.0}
Best Score: 0.621842838382694
Training the XGBoost model...
Training complete.
Training Accuracy: 99.33712121212122

Model Evaluation
Validation Accuracy: 52.84%
Recall: 30.113636363636363
F1 Score: 38.970588235294116
Confusion Matrix:
 [[133  43]
 [123  53]]

Test Evaluation
Test Accuracy: 58.24%
Recall: 29.545454545454547
F1 Score: 41.43426294820717
Confusion Matrix:
 [[153  23]
 [124  52]]

Feature Importance
Correct           0.392108
Response_Time     0.116374
SKT_std           0.092169
EMG_std           0.089631
RESP_std          0.086590
HR_std            0.078642
EDA_std_biopac    0.072347
ACC_std           0.072140
dtype: float32


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=1.0, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=0,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=400, n_jobs=None,
              num_parallel_tree=None, ...)